## LIBRARY

In [1]:
import os
import re
import pandas as pd
import tkinter as tk
from tkinter import ttk
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import matplotlib.pyplot as plt
import networkx as nx
from collections import defaultdict
import math
from collections import Counter

## READING FHIR

In [4]:

directory = 'C:/Users/begon/synthea/output/deneme1'

pattern = re.compile(r'(\"resourceType\": \"(Patient|Encounter|Condition|Observation|Procedure|Medication)\")')

classes = ['Patient', 'Encounter', 'Condition', 'Observation', 'Procedure', 'Medication']

current_resource_type = None
new_resource_type = None
resource_type = None
current_id = None
new_id = None
current_references = []
current_codes = []
display = []
current_patient_id = None
previous_display = None 
previous_code = None 
code_starts_with_LA = False  # Flag to track if the current code starts with "LA"


data = {'ResourceType': [], 'ID': [], 'References': [], 'Codes': [], 'PatientID': []}

counts = {resource_type: 1 for resource_type in classes}

INIT = False

for filename in os.listdir(directory):
    if filename.endswith('.json'):
        with open(os.path.join(directory, filename), 'r', encoding='utf-8') as f:
            for line in f:
                match = pattern.search(line)
                if '"resourceType": "' in line and not match:
                    resource_type = line.split('"')[3]
                    INIT = False
                if match:
                    if match.group(2) in classes:
                        new_resource_type = match.group(2)
                        INIT = True
                        next_line = next(f, None)
                        if next_line and '"id": "' in next_line:
                            new_id = next_line.split('"id": "')[1].split('"')[0]
                if current_resource_type == new_resource_type and '"reference": "urn:uuid:' in line and INIT:
                    reference = line.split(':')[3].split('"')[0]
                    current_references.append(reference)    
                    
                if current_resource_type == new_resource_type and '"code": "' in line and INIT:
                    code = line.split('"code": "')[1].split('"')[0]
                    next_line = next(f, None)
                    if next_line and '"display": "' in next_line:
                        if code.startswith("LA") and previous_display is not None:                    
                            display = previous_display + ': ' + next_line.split('"display": "')[1].split('"')[0]
                            code = previous_code
                            current_codes.append((previous_code, display))
                            print('(code, display)', (current_patient_id, code, display))
                            code_starts_with_LA = True
                        else:
                            display = next_line.split('"display": "')[1].split('"')[0]
                            current_codes.append((code, display))
                            code_starts_with_LA = False
                            #print('(code, display)', (current_patient_id, code, display))
                    # Update previous display only if the code doesn't start with "LA"
                    if not code_starts_with_LA:
                        previous_display = display
                        previous_code = code
                 
                if current_resource_type != new_resource_type  and INIT:
                    if current_resource_type and current_id:
                        data['ResourceType'].append(current_resource_type)
                        data['ID'].append(current_id)
                        if current_resource_type == 'Patient':
                            current_patient_id = current_id
                        data['References'].append(tuple(current_references))
                        data['Codes'].append(tuple(current_codes))
                        data['PatientID'].append(current_patient_id)
                        current_references = []
                        current_codes = []
                    current_resource_type = new_resource_type    
                    current_id = new_id

df = pd.DataFrame(data)
grouped = df.groupby('PatientID')
aggregated_data = grouped.sum()
aggregated_data.to_csv('C:/Users/begon/Desktop/LMI PROJECT/GIT-LMI/Patient1.csv')


(code, display) ('e350e277-a061-8175-a841-2b2df12a6077', '76501-6', 'Within the last year, have you been afraid of your partner or ex-partner?: No')
(code, display) ('e350e277-a061-8175-a841-2b2df12a6077', '93026-3', 'Do you feel physically and emotionally safe where you currently live?: Yes')
(code, display) ('e350e277-a061-8175-a841-2b2df12a6077', '93027-1', 'Are you a refugee?: I choose not to answer this question')
(code, display) ('e350e277-a061-8175-a841-2b2df12a6077', '93028-9', 'In the past year, have you spent more than 2 nights in a row in a jail, prison, detention center, or juvenile correctional facility?: Yes')
(code, display) ('e350e277-a061-8175-a841-2b2df12a6077', '93038-8', 'Stress level: Quite a bit')
(code, display) ('e350e277-a061-8175-a841-2b2df12a6077', '93029-7', 'How often do you see or talk to people that you care about and feel close to (For example: talking to friends on the phone, visiting friends or family, going to church or club meetings)?: 3 to 5 times a

(code, display) ('87be08d6-6189-78b1-bd4e-260d482fd80c', '76501-6', 'Within the last year, have you been afraid of your partner or ex-partner?: No')
(code, display) ('87be08d6-6189-78b1-bd4e-260d482fd80c', '93026-3', 'Do you feel physically and emotionally safe where you currently live?: Yes')
(code, display) ('87be08d6-6189-78b1-bd4e-260d482fd80c', '93027-1', 'Are you a refugee?: No')
(code, display) ('87be08d6-6189-78b1-bd4e-260d482fd80c', '93028-9', 'In the past year, have you spent more than 2 nights in a row in a jail, prison, detention center, or juvenile correctional facility?: Yes')
(code, display) ('87be08d6-6189-78b1-bd4e-260d482fd80c', '93038-8', 'Stress level: I choose not to answer this question')
(code, display) ('87be08d6-6189-78b1-bd4e-260d482fd80c', '93029-7', 'How often do you see or talk to people that you care about and feel close to (For example: talking to friends on the phone, visiting friends or family, going to church or club meetings)?: 5 or more times a week'

(code, display) ('0840b06d-4426-eb63-3c01-af9bcd018556', '76501-6', 'Within the last year, have you been afraid of your partner or ex-partner?: Yes')
(code, display) ('0840b06d-4426-eb63-3c01-af9bcd018556', '93026-3', 'Do you feel physically and emotionally safe where you currently live?: No')
(code, display) ('0840b06d-4426-eb63-3c01-af9bcd018556', '93027-1', 'Are you a refugee?: Yes')
(code, display) ('0840b06d-4426-eb63-3c01-af9bcd018556', '93028-9', 'In the past year, have you spent more than 2 nights in a row in a jail, prison, detention center, or juvenile correctional facility?: Yes')
(code, display) ('0840b06d-4426-eb63-3c01-af9bcd018556', '93038-8', 'Stress level: A little bit')
(code, display) ('0840b06d-4426-eb63-3c01-af9bcd018556', '93029-7', 'How often do you see or talk to people that you care about and feel close to (For example: talking to friends on the phone, visiting friends or family, going to church or club meetings)?: 5 or more times a week')
(code, display) ('084

(code, display) ('0840b06d-4426-eb63-3c01-af9bcd018556', '76501-6', 'Within the last year, have you been afraid of your partner or ex-partner?: No')
(code, display) ('0840b06d-4426-eb63-3c01-af9bcd018556', '93026-3', 'Do you feel physically and emotionally safe where you currently live?: Yes')
(code, display) ('0840b06d-4426-eb63-3c01-af9bcd018556', '93027-1', 'Are you a refugee?: Yes')
(code, display) ('0840b06d-4426-eb63-3c01-af9bcd018556', '93028-9', 'In the past year, have you spent more than 2 nights in a row in a jail, prison, detention center, or juvenile correctional facility?: Yes')
(code, display) ('0840b06d-4426-eb63-3c01-af9bcd018556', '93038-8', 'Stress level: A little bit')
(code, display) ('0840b06d-4426-eb63-3c01-af9bcd018556', '93029-7', 'How often do you see or talk to people that you care about and feel close to (For example: talking to friends on the phone, visiting friends or family, going to church or club meetings)?: Less than once a week')
(code, display) ('0840

(code, display) ('2e1581ff-ec8e-74f9-b253-7fe2d4a4a105', '93031-3', 'In the past year, have you or any family members you live with been unable to get any of the following when it was really needed?: I choose not to answer this question')
(code, display) ('2e1581ff-ec8e-74f9-b253-7fe2d4a4a105', '76437-3', 'Primary insurance: Private insurance')
(code, display) ('2e1581ff-ec8e-74f9-b253-7fe2d4a4a105', '67875-5', 'Employment status - current: Full-time work')
(code, display) ('2e1581ff-ec8e-74f9-b253-7fe2d4a4a105', '82589-3', 'Highest level of education: More than high school')
(code, display) ('2e1581ff-ec8e-74f9-b253-7fe2d4a4a105', '93033-9', 'Are you worried about losing your housing?: No')
(code, display) ('2e1581ff-ec8e-74f9-b253-7fe2d4a4a105', '71802-3', 'Housing status: I have housing')
(code, display) ('2e1581ff-ec8e-74f9-b253-7fe2d4a4a105', '54899-0', 'Preferred language: Language other than English')
(code, display) ('2e1581ff-ec8e-74f9-b253-7fe2d4a4a105', '93034-7', 'Have you 

...

(code, display) ('44294723-2449-04b8-60fe-ad7cab361247', '76501-6', 'Within the last year, have you been afraid of your partner or ex-partner?: Yes')
(code, display) ('44294723-2449-04b8-60fe-ad7cab361247', '93026-3', 'Do you feel physically and emotionally safe where you currently live?: Yes')
(code, display) ('44294723-2449-04b8-60fe-ad7cab361247', '93027-1', 'Are you a refugee?: No')
(code, display) ('44294723-2449-04b8-60fe-ad7cab361247', '93028-9', 'In the past year, have you spent more than 2 nights in a row in a jail, prison, detention center, or juvenile correctional facility?: No')
(code, display) ('44294723-2449-04b8-60fe-ad7cab361247', '93038-8', 'Stress level: A little bit')
(code, display) ('44294723-2449-04b8-60fe-ad7cab361247', '93029-7', 'How often do you see or talk to people that you care about and feel close to (For example: talking to friends on the phone, visiting friends or family, going to church or club meetings)?: 5 or more times a week')
(code, display) ('4429

(code, display) ('2c398646-39ed-d5c8-855b-1fc1f7d751ec', '76501-6', 'Within the last year, have you been afraid of your partner or ex-partner?: No')
(code, display) ('2c398646-39ed-d5c8-855b-1fc1f7d751ec', '93026-3', 'Do you feel physically and emotionally safe where you currently live?: Yes')
(code, display) ('2c398646-39ed-d5c8-855b-1fc1f7d751ec', '93027-1', 'Are you a refugee?: No')
(code, display) ('2c398646-39ed-d5c8-855b-1fc1f7d751ec', '93028-9', 'In the past year, have you spent more than 2 nights in a row in a jail, prison, detention center, or juvenile correctional facility?: No')
(code, display) ('2c398646-39ed-d5c8-855b-1fc1f7d751ec', '93038-8', 'Stress level: Not at all')
(code, display) ('2c398646-39ed-d5c8-855b-1fc1f7d751ec', '93029-7', 'How often do you see or talk to people that you care about and feel close to (For example: talking to friends on the phone, visiting friends or family, going to church or club meetings)?: 5 or more times a week')
(code, display) ('2c39864

(code, display) ('c04ec033-4122-f9de-90ed-44b05e186c7f', '76501-6', 'Within the last year, have you been afraid of your partner or ex-partner?: No')
(code, display) ('c04ec033-4122-f9de-90ed-44b05e186c7f', '93026-3', 'Do you feel physically and emotionally safe where you currently live?: I choose not to answer this question')
(code, display) ('c04ec033-4122-f9de-90ed-44b05e186c7f', '93027-1', 'Are you a refugee?: No')
(code, display) ('c04ec033-4122-f9de-90ed-44b05e186c7f', '93028-9', 'In the past year, have you spent more than 2 nights in a row in a jail, prison, detention center, or juvenile correctional facility?: No')
(code, display) ('c04ec033-4122-f9de-90ed-44b05e186c7f', '93038-8', 'Stress level: A little bit')
(code, display) ('c04ec033-4122-f9de-90ed-44b05e186c7f', '93029-7', 'How often do you see or talk to people that you care about and feel close to (For example: talking to friends on the phone, visiting friends or family, going to church or club meetings)?: 3 to 5 times a 

(code, display) ('7a7f3741-b53e-a2cd-b78e-42a1906c648b', '76501-6', 'Within the last year, have you been afraid of your partner or ex-partner?: No')
(code, display) ('7a7f3741-b53e-a2cd-b78e-42a1906c648b', '93026-3', 'Do you feel physically and emotionally safe where you currently live?: Yes')
(code, display) ('7a7f3741-b53e-a2cd-b78e-42a1906c648b', '93027-1', 'Are you a refugee?: No')
(code, display) ('7a7f3741-b53e-a2cd-b78e-42a1906c648b', '93028-9', 'In the past year, have you spent more than 2 nights in a row in a jail, prison, detention center, or juvenile correctional facility?: No')
(code, display) ('7a7f3741-b53e-a2cd-b78e-42a1906c648b', '93038-8', 'Stress level: Somewhat')
(code, display) ('7a7f3741-b53e-a2cd-b78e-42a1906c648b', '93029-7', 'How often do you see or talk to people that you care about and feel close to (For example: talking to friends on the phone, visiting friends or family, going to church or club meetings)?: 1 or 2 times a week')
(code, display) ('7a7f3741-b53

(code, display) ('2b639a39-6548-02bf-7c48-f7c4ee1da5ad', '76501-6', 'Within the last year, have you been afraid of your partner or ex-partner?: No')
(code, display) ('2b639a39-6548-02bf-7c48-f7c4ee1da5ad', '93026-3', 'Do you feel physically and emotionally safe where you currently live?: Yes')
(code, display) ('2b639a39-6548-02bf-7c48-f7c4ee1da5ad', '93027-1', 'Are you a refugee?: No')
(code, display) ('2b639a39-6548-02bf-7c48-f7c4ee1da5ad', '93028-9', 'In the past year, have you spent more than 2 nights in a row in a jail, prison, detention center, or juvenile correctional facility?: No')
(code, display) ('2b639a39-6548-02bf-7c48-f7c4ee1da5ad', '93038-8', 'Stress level: Not at all')
(code, display) ('2b639a39-6548-02bf-7c48-f7c4ee1da5ad', '93029-7', 'How often do you see or talk to people that you care about and feel close to (For example: talking to friends on the phone, visiting friends or family, going to church or club meetings)?: 5 or more times a week')
(code, display) ('2b639a3

## COMBINE FHIR IN ONE CSV

In [ ]:
for patient_id, group in grouped:
    encounter_references = []
    for index, row in group.iterrows():
        if row['ResourceType'] == 'Encounter':
            patient = row['PatientID']
            encounter_id = row['ID']
            references = [encounter_id]
            codes = [code for code in row['Codes'] if re.match(r'^\d+$|^i$', code)]
            matching_rows = group[group['References'].apply(lambda x: encounter_id in x)]
            matching_codes = [code for codes in matching_rows['Codes'] for code in codes if re.match(r'^\d+$|^i$', code)]
            merged_codes = ','.join(set(codes + matching_codes))
            encounter_references.append({'Patient': patient, 'EncounterID': encounter_id, 'References': references, 'MergedCodes': merged_codes})
    encounter_references_df = pd.DataFrame(encounter_references)
    encounter_references_df.to_csv(f'C:/Users/begon/Desktop/LMI PROJECT/Encounter1/{patient_id}.csv', index=False)

directory = 'C:/Users/begon/Desktop/LMI PROJECT/Encounter1'
dfs = []
for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        csv_path = os.path.join(directory, filename)
        df = pd.read_csv(csv_path)
        selected_columns = df.iloc[:, [1, -1]]
        dfs.append(selected_columns)

combined_df = pd.concat(dfs, ignore_index=True) 
combined_df = combined_df[combined_df['MergedCodes'] != '[]']
combined_df.reset_index(drop=True, inplace=True)
combined_df['MergedCodes'] = combined_df['MergedCodes'].apply(lambda x: x.strip('[]').split(','))
combined_csv_path = 'Encounter_combined_data1.csv'
combined_df.to_csv(combined_csv_path, index=False)

## READ CSV AND CREATE DROP DOWN LIST

In [ ]:
def read_groups_from_csv(filename):
    groups = []
    data = pd.read_csv(filename)
    groups = [eval(code) for code in data['MergedCodes']]
    return groups

combined_csv_path = 'Encounter_combined_data1.csv'
groups = read_groups_from_csv(combined_csv_path)
all_codes = [code for group in groups for code in group]
unique_codes = set(all_codes)
options = [code.strip("[]' ") for code in unique_codes]
print(options)

## A B C D TEST (ignore this part)

In [ ]:
import random

# Predefined groups made up of letters including at least one of A, B, or C
groups = []
for _ in range(10):  # Generate # groups
    group_size = random.randint(1, 3)  # Random group size between 1 and 10
    group = random.sample(['A', 'B', 'C','D'], group_size) #, 'D', 'E', 'F', 'G', 'H', 'I', 'J'
    groups.append(group)

print(groups)  # Print the generated groups

# Get all unique items from the groups
options = list(set(item for group in groups for item in group))

print(options)  # Print the set of all items


## PLOT GRAPH

In [ ]:
def generate_graph(canvas, selected_groups, num_nodes_to_visualize,padding=0.1):
    
    plt.clf() #clear current figure
    
    
    ### Creates the related components and their connections that will be visualized
    G = nx.Graph() # initialize empty indirected graph
    
    for group in selected_groups:
        add_edges_within_group(G, group)  # Add edges within group
    

    # Count occurrences of each node in selected groups
    node_counts = Counter([node for group in selected_groups for node in group])
    print('node_occurrences', node_counts)
        
   
    sorted_nodes = sorted(node_counts, key=node_counts.get, reverse=True)
    print('sorted_nodes',sorted_nodes)
    
    selected_nodes = sorted_nodes[:num_nodes_to_visualize] # related to slider value
    print('selected_nodes',selected_nodes)
    
    subgraph = G.subgraph(selected_nodes)  # only with selected nodes

    
    
    ### Configure visual properties
    if len(subgraph) < 10:
        pos = nx.spring_layout(subgraph)
    else:
        pos = nx.circular_layout(subgraph)
        

    # Count occurrences of edges between nodes in selected groups
    edge_occurrences = Counter()
    edge_labels = {}  # Store labels for each edge

    for group in selected_groups:
        for i in range(len(group)):
            for j in range(i + 1, len(group)):
                node_pair = tuple(sorted([group[i], group[j]]))  # Ensure consistent order for node pair
                if node_pair not in edge_occurrences:
                    edge_occurrences[node_pair] = 0
                    edge_labels[node_pair] = f"{group[i]}-{group[j]}"  # Store label for this edge
                edge_occurrences[node_pair] += 1

    # Calculate edge widths proportional to occurrences
    edge_widths = [edge_occurrences[tuple(sorted(edge))] for edge in subgraph.edges()]

    # Print edge widths alongside corresponding node pairs
    for edge, width in zip(subgraph.edges(), edge_widths):
        node_pair = tuple(sorted(edge))
        print(f"Edge {edge_labels[node_pair]}: {width}")

    
    # Calculate normalization factors based on the number of nodes to visualize
    num_nodes_to_visualize = max(1, min(num_nodes_to_visualize, len(subgraph.nodes())))
    num_edges = len(subgraph.edges())

    if num_nodes_to_visualize > 0:
        node_size_factor = 5000 / num_nodes_to_visualize
    else:
        node_size_factor = 1  # Set a default value to avoid division by zero

    if num_edges > 0:
        edge_width_factor = 10 / num_edges
    else:
        edge_width_factor = 1  # Set a default value to avoid division by zero


    # Normalize node size and edge width
    node_size = [node_counts[n] * node_size_factor for n in subgraph.nodes()]
    edge_widths = [edge_occurrences[tuple(sorted(edge))] * edge_width_factor for edge in subgraph.edges()]

    # Draw nodes and edges with normalized sizes
    nx.draw_networkx_nodes(subgraph, pos, node_size=node_size, node_color="w", edgecolors="k", linewidths=0.5)
    nx.draw_networkx_edges(subgraph, pos, width=edge_widths, edge_color="black", alpha=0.4)
    
        # Add padding to the boundaries of the figure
    xmin, xmax = plt.xlim()
    ymin, ymax = plt.ylim()
    xmargin = (xmax - xmin) * padding
    ymargin = (ymax - ymin) * padding
    plt.xlim(xmin - xmargin, xmax + xmargin)
    plt.ylim(ymin - ymargin, ymax + ymargin)
    
    nx.draw_networkx_labels(subgraph, pos, labels={node: str(node) for node in subgraph.nodes()}, font_size=50/math.log(num_nodes_to_visualize+1,3), font_family='Calibri', font_weight='normal')

    plt.axis("off")
    canvas.draw()


def on_dropdown_select(event):
    
    global selected_groups
    
    selected_code = dropdown.get()
    
    selected_groups = [group for group in groups if selected_code in group]
    print('selected_groups', selected_groups)
    
    num_nodes_to_visualize = len(set([item for sublist in selected_groups for item in sublist]))
    print('num_nodes_to_visualize', num_nodes_to_visualize)
    slider.config(from_=1, to=num_nodes_to_visualize, tickinterval=num_nodes_to_visualize-1)
    
    generate_graph(canvas, selected_groups, 1)

def add_edges_within_group(G, group):
    if isinstance(group, int):
        return
    for i, u in enumerate(group):
        for v in group[:i]:
            if isinstance(u, str) and isinstance(v, str):
                G.add_edge(u, v, weight=1)
            elif isinstance(u, tuple) and isinstance(v, tuple):
                for a in u:
                    for b in v:
                        G.add_edge(a, b, weight=1)

root = tk.Tk()
root.title("Graph Visualization")

selected_groups = []
dropdown = ttk.Combobox(root, values=options)
dropdown.grid(row=0, column=0, padx=10, pady=10)
dropdown.bind("<<ComboboxSelected>>", on_dropdown_select)

initial_num_nodes = 0
slider = tk.Scale(root, from_=0, to=initial_num_nodes, orient=tk.HORIZONTAL, length=200, tickinterval=initial_num_nodes,
                  command=lambda x: generate_graph(canvas, selected_groups, int(x)))
slider.grid(row=0, column=1, padx=10, pady=10)


fig = plt.figure(figsize=(9, 9))
#fig.set_facecolor((0.2, 0.2, 0.2))  # RGB values for a darker grey
canvas = FigureCanvasTkAgg(fig, master=root)
#root.attributes('-alpha', 0.8)  # Adjust alpha value as needed (0.0 to 1.0)
root.configure(bg='darkgrey')
canvas.get_tk_widget().grid(row=1, column=0, columnspan=2)

generate_graph(canvas, [], 0)

root.mainloop()



In [ ]:
import networkx as nx

# Create a graph
G = nx.Graph()

# Add some nodes
G.add_nodes_from([1, 2, 3, 4])

# Add some edges
G.add_edges_from([(1, 2), (1, 3), (2, 3), (3, 4)])

# Create a subgraph
subgraph = G.subgraph([1, 2, 3])

# Get the degree of node 1 in the subgraph
degree_of_node_1 = subgraph.degree(1)

print("Degree of node 1 in the subgraph:", degree_of_node_1)


In [ ]:
import matplotlib.pyplot as plt
import networkx as nx

# Create a graph
G = nx.Graph()

# Add some nodes
G.add_nodes_from([1, 2, 3, 4])

# Add some edges
G.add_edges_from([(1, 2), (1, 3), (2, 3), (3, 4)])

# Create a subgraph
subgraph = G.subgraph([1, 2, 3,4])

# Plot the subgraph
nx.draw(subgraph, with_labels=True, node_color='skyblue', node_size=500, font_size=12, font_color='black')
plt.show()


In [ ]:
subgraph.degree(4)

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
from mpl_toolkits.mplot3d import Axes3D

# Define a function to plot glanzing steel edges with a 3D effect
def plot_glanzing_steel_edges(ax, G, pos, depth=0.1):
    edges = []
    for edge in G.edges():
        x = [pos[edge[0]][0], pos[edge[1]][0]]
        y = [pos[edge[0]][1], pos[edge[1]][1]]
        z = [depth, depth]
        edges.append(list(zip(x, y, z)))
    for edge in edges:
        ax.plot(*edge, color='silver', linewidth=2)

# Create a graph
G = nx.Graph()

# Add nodes
G.add_nodes_from([1, 2, 3, 4, 5, 6])

# Add edges
G.add_edges_from([(1, 2), (1, 3), (2, 3), (3, 4), (4, 5), (4, 6), (5, 6)])

# Specify node positions in 2D space
pos = {1: (1, 1), 2: (2, 2), 3: (3, 3), 4: (4, 4), 5: (5, 5), 6: (6, 6)}

# Draw the graph
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')

# Call function to plot glanzing steel edges with a 3D effect
plot_glanzing_steel_edges(ax, G, pos)

# Set axis labels
ax.set_xlabel('X', fontsize=12)
ax.set_ylabel('Y', fontsize=12)
ax.set_zlabel('Depth', fontsize=12)

# Set title
plt.title('Glanzing Steel Edges', fontsize=14)

# Show the graph
plt.show()
